### Basic Operations 

#### Accessing and Modifying pixel values

In [ ]:
import numpy as np
import cv2
import utils

In [ ]:
# loading images to memory
img = cv2.imread('../images/image1.jpg')
assert img is not None

print(img.size) # number of pixels
print(img.dtype) #
print(img.shape) # 
print(img[100,100]) # [230 210 222] Blue, Green, Red (bgr)
utils.display_img(img)

In [ ]:
# selecting a region in the image
face = img[700:900,800:1000]
img[1500:1700, 500:700] = face
utils.display_img(img)

In [ ]:
# draw 1/5 of pixels black
utils.draw_noise(img)
utils.display_img(img)

#### Splitting and Merging Image Channels

In [ ]:
img = cv2.imread('../images/image1.jpg')

b,g,r = cv2.split(img)

print(np.array_equal(b, img[:,:,0]))
print(np.array_equal(g, img[:,:,1]))
print(np.array_equal(r, img[:,:,2]))

g[:] = 0

img = cv2.merge((b,g,r))
# faster to do img[:,:,1] = 0
utils.display_img(img)

#### Making Borders for Images

In [ ]:
img = cv2.imread('../images/image2.jpg')
img2 = cv2.imread('../images/image1.jpg')

face = img2[700:900,800:1000]
img[800:1000,350:550] = face

constant = cv2.copyMakeBorder(img, 15, 15, 15, 15, cv2.BORDER_CONSTANT, value=[255,0,0])

utils.display_img(constant)

### Arithmetic Operations

#### Image Addition

In [ ]:
img1 = cv2.imread('../images/image1.jpg')
img2 = cv2.imread('../images/image2.jpg')

print(img1.shape, img2.shape)

# resize img1 to match img2 dimensions for addition
x_offset, y_offset = 500, 0
rows, cols, channels = img2.shape
img1 = img1[0+x_offset:x_offset+rows,0+y_offset:y_offset+cols]

# Remove blue channel from img1
img1[:,:,0] = 0

# display pictures side by side
utils.display_mul_img([img1, img2])

# display addition result
utils.display_img(cv2.add(img1, img2), "img1 + img2")

#### Image Blending

In [ ]:
img1 = cv2.imread('../images/image1.jpg')
img2 = cv2.imread('../images/image2.jpg')

# resize img1 to match img2 dimensions for blending
x_offset, y_offset = 500, 0
rows, cols, channels = img2.shape
img1 = img1[0+x_offset:x_offset+rows,0+y_offset:y_offset+cols]

# blend two images
# dst = alpha * img1 + beta * img2 + gamma
img3 = cv2.addWeighted(img1, 0.5, img2, 0.5, 0)

utils.display_mul_img([img1, img2, img3], ["img1", "img2", "img3"])

#### Bitwise Operations

In [ ]:
# Overlay a non-rectangular logo onto an image
# Using masking, thresholding, and bitwise operations

img1 = cv2.imread('../images/image1.jpg')
img2 = cv2.imread('../images/opencv-logo.png')

utils.display_mul_img([img1, img2], ["input1", "input2"])

rows, cols, channels = img2.shape
roi = img1[0:rows, 0:cols]

# replace color [190, 154, 80] by [0, 0, 0]
# create a mask for the background color ([190, 154, 80] in BGR) and replace it with black
target_color = np.array([190, 154, 80])
# mask = cv2.inRange(img2, target_color, target_color)
# img2[mask > 0] = [0, 0, 0]

# create a logo mask
img2gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
# ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
mask_inv = cv2.inRange(img2, target_color, target_color)
mask = cv2.bitwise_not(mask_inv)

# n xor n = 0
# use mask_inv to black out logo area in roi, allowing clean addition with logo foreground
img1_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
img2_fg = cv2.bitwise_and(img2, img2, mask=mask)
dst = cv2.add(img1_bg, img2_fg)

utils.display_mul_img([img2, img2gray, mask, mask_inv])
utils.display_mul_img([roi, img1_bg, img2_fg, dst])

# copy the combined result back to the original image region
img1[0:rows, 0:cols] = dst

utils.display_img(img1, "output")

### Performance Measurement and Improvement

#### Measuring Performance

In [ ]:
img1 = cv2.imread('../images/image1.jpg', 0)

e1 = cv2.getTickCount()
# task start
for i in range(5,49,2):
    img1 = cv2.medianBlur(img1,i)
# task end
e2 = cv2.getTickCount()
time = (e2 - e1)/ cv2.getTickFrequency()
print(time)

In [ ]:
# cv2.countNonZero is faster than np.count_nonzero
%timeit z = cv2.countNonZero(img1)

In [ ]:
%timeit z = np.count_nonzero(img1)

#### Performance Optimization Techniques
First implement and verify functionality, then profile to identify bottlenecks, and optimize critical sections.

- Avoid loops whenever possible
- Maximize vectorization using NumPy and OpenCV built-in operations